In [4]:
import boto3
import pandas as pd
import numpy as np
import io
import sys
import os
import seaborn as sns
import sqlparse

# local libraries
sys.path.append(os.path.abspath(os.path.join(sys.path[0], '..', 'src', 'lib')))
#sys.path.append('/mnt/src/lib') # Domino
import market_maker_training

# options
pd.options.display.max_rows = 100
pd.options.display.max_columns = 1000
pd.options.mode.chained_assignment = None
%matplotlib inline

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
coin_pair_dict = {'target':'btcusdt'}
feature_minutes_list = [5]
trade_window_list=[6]

### Print SQL

In [12]:
mm_training = market_maker_training.BinanceTraining(coin_pair_dict, feature_minutes_list, trade_window_list)
bsql = mm_training.training_data_sql
bsql = bsql.replace('\n','')
print(sqlparse.format(bsql, reindent=True, keyword_case='upper'))

WITH target_btcusdt_end_orderbook AS
  (SELECT trade_minute - 1 AS lag_trade_minute,
          *
   FROM binance.orderbook
   WHERE coin_pair = 'btcusdt'
   ORDER BY trade_minute DESC),
     target_btcusdt_beg_orderbook AS
  (SELECT *
   FROM binance.orderbook
   WHERE coin_pair = 'btcusdt'
   ORDER BY trade_minute DESC),
     target_btcusdt_candlesticks AS
  (SELECT *
   FROM binance.candledicks c
   WHERE coin_pair = 'btcusdt'
   ORDER BY trade_minute DESC) ,
     target_btcusdt_features AS
  (SELECT c.close_datetime AS btcusdt_trade_close_datetime ,
          extract(isodow
                  FROM c.close_datetime) AS trade_day_of_week ,
          date_part('hour', c.close_datetime) AS trade_hour ,
          c.close_datetime::date - CURRENT_DATE AS days_old ,
          c.trade_minute AS btcusdt_trade_minute ,
          quote_asset_volume AS btcusdt_quote_asset_volume ,
          taker_sell_volume_percentage * 100 AS btcusdt_taker_sell_volume_perc_of_total ,
          trade_count AS b